Here basically we will be trying to read midi files and see if we can get the note names


In [12]:
import mido
import pretty_midi
from mido import MidiFile
import pandas as pd
import numpy as np

In [2]:
midi = MidiFile("../data/midi/0/100680.mid", clip=True)

In [3]:
def get_time_signature(midi):
    for track in midi.tracks:
        for msg in track:
            if msg.is_meta and msg.type == "time_signature":
                numerator = msg.numerator
                denominator = msg.denominator
                return numerator, denominator
    return 4, 4  # default


def get_key_signature(track):
    for msg in track:
        if msg.is_meta and msg.type == "key_signature":
            key = msg.key
            return key
    return "None"


def get_measure_length(midi):
    n, d = get_time_signature(midi)
    beat_length = midi.ticks_per_beat
    measure_length = n * beat_length
    return beat_length, measure_length

In [4]:
from mido.midifiles.midifiles import DEFAULT_TICKS_PER_BEAT


class CutomMidiFile(mido.MidiFile):
    def __init__(
        self,
        filename=None,
        file=None,
        type=1,
        ticks_per_beat=...,
        charset="latin1",
        debug=False,
        clip=False,
        tracks=None,
    ):
        super().__init__(
            filename, file, type, ticks_per_beat, charset, debug, clip, tracks
        )

    def properties(self):
        pass

    def get_time_signature(self):
        for track in self.tracks:
            for msg in track:
                if msg.is_meta and msg.type == "time_signature":
                    numerator = msg.numerator
                    denominator = msg.denominator
                    return numerator, denominator
        return 4, 4  # default

    def get_key_signature(self, track_idx: int = None):
        if track_idx:
            lookup = self.tracks[track_idx]
        else:
            lookup = self
        for msg in lookup:
            if msg.is_meta and msg.type == "key_signature":
                key = msg.key
                return key
        return "None"

    def get_measure_length(self):
        n, d = self.get_time_signature()
        beat_length = self.ticks_per_beat
        measure_length = n * beat_length
        return beat_length, measure_length

    def n_tracks(self):
        return len(self.tracks)

In [5]:
def parse_midi(midi, track):
    note_msgs = []
    for msg in midi.tracks[track]:
        if msg.type == "note_on":
            note_msgs.append([msg, pretty_midi.note_number_to_name(msg.note)])
    return note_msgs

numerator -> number of beats in a measure
x -> number of ticks per beat

number of ticks per measure = ? = n \* x


In [21]:
sargam = np.array(["S", "r", "R", "g", "G", "M", "m", "d", "D", "n", "N"])
lower = np.array([f",{_}" for _ in sargam])
upper = np.array([f"{_}'" for _ in sargam])
western = np.array(["A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#"])

In [28]:
for i in range(60, 72):
    print(pretty_midi.note_number_to_name(i)[:-1])

C
C#
D
D#
E
F
F#
G
G#
A
A#
B


In [11]:
beats_per_measure = 4
ticks_per_beat = midi.ticks_per_beat
ticks_per_measure = ticks_per_beat * beats_per_measure
song = []
measure = []
for i, msg in enumerate(midi.tracks[0]):
    if msg.type == "time_signature":
        beats_per_measure = msg.numerator
        ticks_per_measure = ticks_per_beat * beats_per_measure
    else:
        try:
            nxt_msg = midi.tracks[0][i+1]
        except:
            break
        if nxt_msg.type == 'end_of_track':
            note = pretty_midi.note_number_to_name(msg.note)
            measure.append(note)
            song.append(measure)
            print("track end")
            break
        if nxt_msg.note == msg.note:
            m
        if msg.velocity > 0:
            
        
            

MetaMessage('key_signature', key='E', time=0)
MetaMessage('key_signature', key='D', time=1)
MetaMessage('key_signature', key='A', time=1)
MetaMessage('key_signature', key='A', time=1)
MetaMessage('key_signature', key='C', time=1)
MetaMessage('key_signature', key='A', time=1)
MetaMessage('key_signature', key='A', time=1)
MetaMessage('key_signature', key='E', time=0)
MetaMessage('key_signature', key='D', time=0)


In [34]:
midi.tracks[0][-2].velocity

0